In [148]:
# import modules required to open the file and perform analysis 

import os
import pandas as pd 
import json 
import numpy as np
from IPython.display import display



# Adjusting the view for results displayed below 

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


with open("/Users/pooja/Documents/Pandas HW_ House of Pymoli/purchase_data.json") as data_file:
    data = json.load(data_file)
    df = pd.DataFrame(data)
    
    df.columns = [c.replace(' ', '_') for c in df.columns]
    #print (df.head())
    
    display(df.head()) 
    
    
    

height has been deprecated.



,Age,Gender,Item_ID,Item_Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [2]:
# Player Count

print("The total number of players are:"+ str(df.SN.nunique()))


The total number of players are:573


In [3]:
# Number of unique items, average purchase price , total number of purchases and total revenue 

unique_items=df.Item_Name.nunique()
Avg_purchase_price=df['Price'].mean()
total_purchases=df.Item_ID.count()
revenue=df['Price'].sum()

df_result = pd.DataFrame({'Unique_Items': [unique_items], 'Average_Purchase_Price': [Avg_purchase_price],
                          'Total_Number_Purchases':[total_purchases],'Total_Revenue':[revenue]})

print(df_result)



   Average_Purchase_Price  Total_Number_Purchases  Total_Revenue  Unique_Items
0                2.931192                     780        2286.33           179


In [149]:
#Gender Demographics 

# created a new Data Frame that groups by Screen Name(SN) and Gender (This is to get unique screen names and count gender acordingly)

counts = df.groupby(['SN','Gender'], as_index=False).size().unstack(fill_value=0).reset_index()

# Retrieve the number of rows where the male column has a value 
male_count=counts[(counts['Male']>0)].count()

MC=(male_count['Male'])

# Retrieve the number of rows where the Female column has a value 

female_count=counts[(counts['Female']>0)].count()

FC=(female_count['Female'])

# Retrieve the total of unique screen names and added them ( this is to know the remainder which makes the others / Non Disclosed gender count)

Total=df.SN.nunique()

Known=(MC+FC)

Others=Total-Known

# Calculate the percentages of gender Types 

MP=(MC/Total)*100
FP=(FC/Total)*100
OP=(Others/Total)*100

# creating a list to re name the index so that they are more descriptive 

new_index=['Total','Percentage']

# Creating a Data Frame to display the results 

gender_result=pd.DataFrame({'Male':[MC,MP],'Female':[FC,FP],'Others/Non-Disclosed':[Others,OP]},index=new_index)

display (gender_result)





,Female,Male,Others/Non-Disclosed
Total,100.00,465.00,8.00
Percentage,17.45,81.15,1.40


In [151]:
#Purchasing Analysis by Gender 

#Purchase Count
#Average Purchase Price
#Total Purchase Value
#Normalized Totals 

#gender_ana = df.groupby(['Gender'], as_index=False).size().unstack(fill_value=0).reset_index()

gender_ana=df.groupby(['Gender'], as_index=False)['Price'].sum()
# Total Purchase value by gender 

Avg_Purchase_Female=gender_ana.iloc[0,1]/FC
Avg_Purchase_Male=gender_ana.iloc[1,1]/MC
Avg_Purchase_Other=gender_ana.iloc[2,1]/Others


Avg_Price= [Avg_Purchase_Female,Avg_Purchase_Male,Avg_Purchase_Other]

gender_ana['Normalized_Total']=Avg_Price


gender_ana1= df.groupby(['Gender'],as_index=False)['Item_ID'].count()


gender_ana['Purchase_count']=gender_ana1['Item_ID']

gender_ana['Average_Price']=gender_ana['Price']/gender_ana['Purchase_count']

gender_ana






,Gender,Price,Normalized_Total,Purchase_count,Average_Price
0,Female,382.91,3.83,136,2.82
1,Male,"1,867.68",4.02,633,2.95
2,Other / Non-Disclosed,35.74,4.47,11,3.25


In [144]:
# Create bins to categorize the age 

bins = [0,10,14,19,24,29,34,40,100]


# Create the names for the four bins
group_names = ['>10','10-14', '15-19', '20-24','25-29','30-34','35-40','<40']

#pd.cut(df["Age"], bins, labels=group_names)

df["Age Group"] = pd.cut(df["Age"],bins, labels=group_names)




age_ana= df.groupby(['Age Group'],as_index=False)['Item_ID'].count()

age_ana1= df.groupby(['Age Group'],as_index=False)['Price'].sum()


#age_ana1['Avg_Price_Age']=(age_ana1['Price']/age_ana1['Price'].sum())*100

#age_ana['Price1']=df.groupby(['Age Group'],as_index=False)['Price'].sum()


result_age_demographics = pd.merge(age_ana, age_ana1, on='Age Group')


pd.options.display.float_format = '{:20,.2f}'.format

result_age_demographics['Avg_Purchase']=result_age_demographics['Price']/result_age_demographics['Item_ID']

result_age_demographics['Normalized_Total']=result_age_demographics['Item_ID'].sum()/result_age_demographics['Item_ID']

result_age_demographics







,Age Group,Item_ID,Price,Avg_Purchase,Normalized_Total
0,>10,32,96.62,3.02,24.38
1,10-14,31,83.79,2.70,25.16
2,15-19,133,386.42,2.91,5.86
3,20-24,336,978.77,2.91,2.32
4,25-29,125,370.33,2.96,6.24
5,30-34,64,197.25,3.08,12.19
6,35-40,56,164.51,2.94,13.93
7,<40,3,8.64,2.88,260.00


In [117]:
top_ana=df.groupby(['SN'],as_index=False)['Price'].sum()

top_ana2=top_ana.sort_values('Price',ascending=False).head()

top_ana1= df.groupby(['SN'],as_index=False)['Item_ID'].count()

#top_ana1

top_ana3 = pd.merge(top_ana2, top_ana1, on='SN')

top_ana3['Avg_Price_Top5']=top_ana3['Price']/top_ana3['Item_ID']

top_ana3



,SN,Price,Item_ID,Avg_Price_Top5
0,Undirrala66,17.06,5,3.41
1,Saedue76,13.56,4,3.39
2,Mindimnya67,12.74,4,3.18
3,Haellysu29,12.73,3,4.24
4,Eoda93,11.58,3,3.86


In [152]:
# most popular items  


df1=df.groupby(['Item_Name','Item_ID','Price']).count()



df1=df1.sort_values('Age',ascending=False)



df1=df1.rename(columns = {'Age':'Purchase_Count'}).head()

df1 = df1.drop(['Gender','SN'],1)

df1=df1.reset_index()

df1['Total Purchase Value']=df1['Purchase_Count']*df1['Price']

display(df1)




,Item_Name,Item_ID,Price,Purchase_Count,Total Purchase Value
0,"Betrayal, Whisper of Grieving Widows",39,2.35,11,25.85
1,Arcane Gem,84,2.23,11,24.53
2,Retribution Axe,34,4.14,9,37.26
3,Trickster,31,2.07,9,18.63
4,Serenity,13,1.49,9,13.41


In [153]:
# Most profitable Items 

df1=df.groupby(['Item_Name','Item_ID','Price']).count()



#df1=df1.sort_values('Age',ascending=False)



df1=df1.rename(columns = {'Age':'Purchase_Count'})

df1 = df1.drop(['Gender','SN'],1)

df1=df1.reset_index()

df1['Total Purchase Value']=df1['Purchase_Count']*df1['Price']

#print(df1)

df2=df1.sort_values('Total Purchase Value',ascending=False).head()

display(df2)








,Item_Name,Item_ID,Price,Purchase_Count,Total Purchase Value
115,Retribution Axe,34,4.14,9,37.26
135,Spectral Diamond Doomblade,115,4.25,7,29.75
98,Orenmir,32,4.95,6,29.70
127,Singed Scalpel,103,4.87,6,29.22
137,"Splitter, Foe Of Subtlety",107,3.61,8,28.88
